<a href="https://colab.research.google.com/github/ipeirotis/sql_autograding/blob/main/Few_shot_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install -q openai
!pip install -q google-cloud-secret-manager
!pip3 install -U -q PyMySQL sqlalchemy sql_magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 9.1 MB/s eta 0:00:00


In [ ]:
from google.colab import auth

# Login using the account that has access to the Google project
# in order to access the resources for the project
auth.authenticate_user()

In [ ]:
from google.cloud import secretmanager


def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload of the given secret version and return it.

    Args:
        project_id (str): Google Cloud project ID.
        secret_id (str): ID of the secret to access.
        version_id (str): ID of the version to access.
    Returns:
        str: The secret version's payload, or None if
        the version does not exist.
    """
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    return response.payload.data.decode("UTF-8")


openai_key = access_secret_version("sql-autograding", "openai-gpt4-32k", "latest")

In [ ]:
import openai

openai.api_key = openai_key

In [ ]:
from google.colab import files
import io
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import gcsfs
# load the cleaned dataset
fs = gcsfs.GCSFileSystem(project="sql_autograding")
with fs.open("gs://sql_autograding/cleaned_response.csv") as f:
    data = pd.read_csv(f)

In [ ]:
data = data.drop(columns="Unnamed: 0")
data.head()

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text

## Further preprocessing

In [ ]:
db_list = [
    "flights",
    "imdb",
    "music",
    "restaurants",
    "facebook",
    "northwind",
    "collisions",
]


def db_schema(db_name):
    """
    Connects to a database and returns the schema of each table in the database.

    The function connects to a specific database using SQLAlchemy. It then retrieves the list of tables in
    the database and for each table, it queries the schema (i.e., the list of fields/columns) and stores it
    in a dictionary. The function returns a list of such dictionaries, with each dictionary representing a table
    and its corresponding schema.

    Args:
        db_name (str): The name of the database to connect to and retrieve schemas from.

    Returns:
        list: A list of dictionaries, with each dictionary containing the name of a table as the key and a
              list of its fields as the value.

    Example:
        >>> db_schema('flights')
        {'m_airports': ['airport', 'state', 'state_name']},
        {'m_ticket_prices': ['origin', 'dest', 'carrier', 'fare', 'fare_per_mile','passengers', 'distance']
        ...
        },
    """

    student_password = access_secret_version(
        "sql-autograding", "db_student_password", "1"
    )

    conn_string = (
        "mysql+pymysql://{user}:{password}@{host}/{db}?charset=utf8mb4".format(
            host="db.ipeirotis.org",
            user="student",
            password=student_password,
            db=db_name,
            encoding="utf8mb4",
        )
    )
    engine = create_engine(conn_string)

    with engine.begin() as conn:
        tables = pd.read_sql_query(sql=text("show tables"), con=conn)
        tables = tables.iloc[:, -1].tolist()

        schema_list = []
        for t in tables:
            d = pd.read_sql_query(sql=text(f"describe {t}"), con=conn)
            table_schema = {t: d.loc[:, "Field"].tolist()}
            schema_list.append(table_schema)
    return schema_list

In [ ]:
# generates the schema for each of the databases in the dataset
flights_schema = db_schema("flights")
imdb_schema = db_schema("imdb")
music_schema = db_schema("music")
restaurants_schema = db_schema("restaurants")
facebook_schema = db_schema("facebook")
northwind_schema = db_schema("northwind")
collisions_schema = db_schema("collisions")

print(facebook_schema)

In [ ]:
# creates a dictionary that maps each schema to each database and add the schema as a new column in the dataframe
schema_mapping = {
    "flights": flights_schema,
    "imdb": imdb_schema,
    "music": music_schema,
    "restaurants": restaurants_schema,
    "facebook": facebook_schema,
    "northwind": northwind_schema,
    "collisions": collisions_schema,
    "['northwind', 'flights']": (northwind_schema, flights_schema),
}

data["schema"] = data["Database"].map(schema_mapping)
data.head()

In [ ]:
from bs4 import BeautifulSoup

# Create a function to clean up a single text string
def clean_html_content(text):
    if pd.isna(text):
        return text
    if not isinstance(text, str):
        text = str(text)
    try:
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text()
    except:
        return text


# Apply the function to every text column in your DataFrame
for col in data.columns:
    if col in ("Database", "schema"):
        continue
    if data[col].dtype == object:  # if the column is a text column
        data[col] = data[col].apply(clean_html_content)

## Generating Mega Prompt
===============================

a. We are working with the XXX database, which has the following tables: artist(id, first_name, last_name) album(id, artist_id, name) track....

b. The request to the student is "Fetch all the tracks for user X"

c. The model answer is ..... (note that we may have multiple correct answers, the model answer is just an example)

d. The student answer was .....

e. Previously, students have submitted these answers and got back these responses and grades:

e1. submission: ..... , grade: ..... , feedback

e2. submission: ..... , grade: ..... , feedback

e3. submission: ..... , grade: ..... , feedback

Please provide a grade and feedback for the student

===============================

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from typing import List


def generate_submissions_string(
    train_df: pd.DataFrame, submission_size: int = None
) -> str:
    """
    Generate a string that concatenates student submissions up to the given size.

    Args:
        train_df (pd.DataFrame): The DataFrame containing the training data.
        submission_size (int, optional): The number of submissions to include in the string. If None, include all submissions.

    Returns:
        str: A string containing the concatenated submissions.
    """
    submissions = ""
    for i, row in train_df.iterrows():
        if submission_size is not None and i >= submission_size:
            break
        submission = row["InputUserAnswer"]
        grade = row["Score"]
        full_grade = row["full_score"]
        feedback = row["feedback"]
        submissions += f"e{i}. submission: {submission}, grade: {grade}/{full_grade}, feedback: {feedback} \\n\\n"
    return submissions


def generate_mega_prompt_for_id(
    df: pd.DataFrame, id: str, submission_size: int = None
) -> str:
    """
    Generate a mega prompt for a specific QuestionId.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        id (str): The QuestionId to generate the mega prompt for.
        submission_size (int, optional): The number of submissions to include in the mega prompt. If None, include all submissions.

    Returns:
        str: The mega prompt.
    """
    temp = df[df["QuestionId"] == id]

    # If there's only one row or less, return None or handle it in a special way
    if len(temp) == 0:
      return None
    if len(temp) == 1:
      # return the only row as test answer
      temp = temp.reset_index()
      submissions = ''
      db = temp.loc[0, "Database"]
      tables = temp.loc[0, "schema"]
      question = temp.loc[0, "Question"]
      key = temp.loc[0, "AnswerKey"]

      test = temp.loc[0, "InputUserAnswer"]

      mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade (out of 10 points) and feedback for the student
        """
      return mega_p

    # making test size 1
    train_df, test_df = train_test_split(temp, test_size=1, random_state=1234)
    train_df = train_df.reset_index()
    test_df = test_df.reset_index()

    submissions = generate_submissions_string(train_df, submission_size)

    db = train_df.loc[0, "Database"]
    tables = train_df.loc[0, "schema"]
    question = train_df.loc[0, "Question"]
    key = train_df.loc[0, "AnswerKey"]

    test = test_df.loc[0, "InputUserAnswer"]

    mega_p = f"""
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

      Please provide a grade and feedback for the student
      """
    return mega_p


def generate_mega_prompts(
    df: pd.DataFrame, submission_size: int = None
) -> pd.DataFrame:
    """
    Generate a DataFrame of mega prompts for each unique QuestionId in the given DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        submission_size (int, optional): The number of submissions to include in each mega prompt. If None, include all submissions.

    Returns:
        pd.DataFrame: A DataFrame containing the mega prompts.
    """
    result = []
    qids = df["QuestionId"].unique()

    for id in qids:
        mega_p = generate_mega_prompt_for_id(df, id, submission_size)
        new_row = {"QuestionId": id, "MegaPrompt": mega_p}
        result.append(new_row)

    mega_df = pd.DataFrame(result)

    return mega_df

In [ ]:
# for all questions each with all submissions => new df
mega_df_all = generate_mega_prompts(data)
mega_df_all.head()

In [ ]:
# Answering Mega Prompt Using GPT 3.5 model
# Deal with rate limit of GPT 3.5-turbo model
from tenacity import(retry, stop_after_attempt, wait_random_exponential)
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT3_generation_with_backoff(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
# Answering Mega Prompt Using GPT 4-32k model
def GPT4_generation(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4-32k",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
# A function that takes QuestionId and submission size as input, generating n-size mega prompt with a random test case and printing out GPT4 answer
def test_shots(qid, n):
    df = data[data["QuestionId"] == qid]
    df_n = generate_mega_prompts(df, n)
    print(f"{n} submissions:")
    print(GPT4_generation(df_n.loc[0, "MegaPrompt"]))

In [ ]:
for i in range(0, 21, 5):
    test_shots("5,168,443", i)

In [ ]:
from tqdm import tqdm

In [ ]:
# Generate GPT4 answers for all QuestionIds (if needed)
for i, p in tqdm(mega_df_all.iterrows()):
    p["feedback"] = GPT4_generation(p["MegaPrompt"])

mega_df_all.head()

In [ ]:
mega_df_all["feedback"] = mega_df_all["MegaPrompt"].apply(GPT4_generation)
mega_df_all.head()

## DataFrame with Zero-Shot Prompts

In [ ]:
df_new = data.copy()
df_new.head()

In [ ]:
# A function that parses grade and feedback from GPT responses
def parse_grade_feedback(text):
  colon_pos = text.find(":")
  divider_pos = text.find("/")

  grade = text[colon_pos+2:divider_pos]

  feedback_start = text.find("Feedback: ")
  if feedback_start != -1:
      feedback = text[feedback_start + len("Feedback: "):]
  else:
      feedback = None
  return grade, feedback

In [ ]:
lst = []
for i,r in df_new.iterrows():
  # generate zero-shot mega prompt for each row of the dataframe
  lst.append(generate_mega_prompts(r.to_frame().T,0).loc[0,'MegaPrompt'])
# append mega prompts as a new column to the dataframe
df_new['0_mp'] = lst
df_new.head()

In [ ]:
tqdm.pandas()
# generate GPT4-32k responses for each row and parse grade and feedback as new columns
df_new['4_answer'] = df_new['0_mp'].progress_apply(GPT4_generation)
df_new['4_grade'], df_new['4_feedback'] = zip(*df_new['4_answer'].apply(parse_grade_feedback))
df_new.head()

In [ ]:
# generate GPT3.5-turbo responses for each row and parse grade and feedback as new columns
df_new['3_answer'] = df_new['0_mp'].progress_apply(GPT3_generation_with_backoff)
df_new['3_grade'], df_new['3_feedback'] = zip(*df_new['3_answer'].apply(parse_grade_feedback))
df_new.head()

In [ ]:
# save the dataframe with zero-shot mega prompts and GPT responses for evaluation
df_new.to_csv('completion.csv')

## DataFrame with Few-Shot Prompts

In [ ]:
with fs.open("gs://sql_autograding/completion.csv") as f:
    data = pd.read_csv(f)

In [ ]:
data = data.drop(columns="Unnamed: 0")
data.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,...,full_score,feedback,schema,0_mp,4_answer,4_grade,4_feedback,3_answer,3_grade,3_feedback
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,...,10.0,The feedback from the TA was Results for ident...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 9/10\n\nFeedback: Your answer is almost...,9,Your answer is almost perfect. You have correc...,Grade: 7/10\n\nFeedback: \n- The student's que...,7,\n- The student's query is mostly correct and ...
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,...,10.0,The feedback from the TA was -3 pts for lack f...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 8/10\n\nFeedback: Your SQL query is mos...,8,Your SQL query is mostly correct and it shows ...,Grade: 8/10\n\nFeedback: \n- The student corre...,8,\n- The student correctly joins the Orders and...
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,...,7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 9/10\n\nFeedback: Your SQL query is alm...,9,Your SQL query is almost perfect. You have cor...,Grade: 7/10\n\nFeedback: \n- The query provide...,7,\n- The query provided by the student is almos...
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,...,10.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Excellent job! Your ...,10,Excellent job! Your query correctly retrieves ...,Grade: 8/10\n\nFeedback: \n- The query provide...,8,\n- The query provided by the student is corre...
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,...,7.5,NaN,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwi...,Grade: 10/10\n\nFeedback: Great job! Your SQL ...,10,Great job! Your SQL query correctly retrieves ...,Grade: 8/10\n\nFeedback: \n- The query provide...,8,\n- The query provided by the student is corre...


In [ ]:
# Remove the zero-shot columns
data = data.drop(columns=['0_mp','4_answer','4_grade','4_feedback','3_answer','3_grade','3_feedback'])
data.head()

,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
0,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,729",Hard question. Requires use of subqueries and ...,Long Answer Question,"SELECT \n A.CustomerID,\n A.CompanyName,...",10.0,ab3864,Results for identical to the answer key becaus...,"WITH total_spent AS (\nSELECT C.CustomerID, C....",True,"['northwind', 'flights']",10.0,The feedback from the TA was Results for ident...,"([{'Categories': ['CategoryID', 'CategoryName'..."
1,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,713","For each order, show the customerID who placed...",Long Answer Question,"Select O.OrderID, O.CustomerID, Count(D.Produ...",5.5,ab3864,-3 pts for lack for HAVING\n-1 pts for using t...,"SELECT O.OrderID, O.CustomerID\n , SUM(D.Qu...",True,"['northwind', 'flights']",10.0,The feedback from the TA was -3 pts for lack f...,"([{'Categories': ['CategoryID', 'CategoryName'..."
2,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,711",We want to identify the ship countries with th...,Long Answer Question,"use northwind;\nSELECT \n ShipCountry, AVG(...",7.0,ab3864,"It is ""LIMIT 3"" not ""LIMIT = 3""","SELECT ShipCountry, ROUND(AVG(Freight),2) AS A...",True,"['northwind', 'flights']",7.5,"The feedback from the TA was It is ""LIMIT 3"" n...","([{'Categories': ['CategoryID', 'CategoryName'..."
3,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"Select C.CompanyName, C.CustomerID, sum(case w...",10.0,ab3864,NaN,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."
4,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,709",Show the date/time of the first order ever dat...,Long Answer Question,use northwind;\nSELECT \n min(OrderDate) as...,7.5,ab3864,NaN,SELECT MIN(OrderDate) AS earliestOrderDate\nFR...,True,"['northwind', 'flights']",7.5,NaN,"([{'Categories': ['CategoryID', 'CategoryName'..."


## Mega-Prompt Generation

In [ ]:
# A function that generates few-shot mega-prompts for a row given the expected number of submissions
from numpy import row_stack
def generate_fewshot_prompt_for_row(
    test, submission_size: int = None
) -> str:
    """
    Generate a mega prompt for a specific QuestionId.

    Args:
        test (pd.DataFrame): The DataFrame - this row.
        submission_size (int, optional): The number of submissions to include in the mega prompt. If None, include all submissions.

    Returns:
        str: The mega prompt.
    """
    temp = data[data["QuestionId"] == test['QuestionId']]

    # If there's only one row or less, return None or handle it in a special way
    if len(temp) == 0:
      return None
    if len(temp) == 1:
      # return the only row as test answer
      temp = temp.reset_index()
      submissions = ''
      db = temp.loc[0, "Database"]
      tables = temp.loc[0, "schema"]
      question = temp.loc[0, "Question"]
      key = temp.loc[0, "AnswerKey"]

      test = temp.loc[0, "InputUserAnswer"]

      mega_p = f"""
        a. We are working with the {db} database, which has the following tables: {tables}

        b. The request to the student is "{question}"

        c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

        d. The student answer was {test}

        e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

        Please provide a grade (out of 10 points) and feedback for the student
        """
      return mega_p

    # skip the current row
    temp = temp[temp['NetId']!=test['NetId']]
    if submission_size==None:
      train_df = temp.copy().reset_index()
    else:
      # randomly sampling certain size of training rows for the same QuestionId
      train_df = temp.sample(n=submission_size).reset_index()

    submissions = generate_submissions_string(train_df)

    db = train_df.loc[0, "Database"]
    tables = train_df.loc[0, "schema"]
    question = train_df.loc[0, "Question"]
    key = train_df.loc[0, "AnswerKey"]
    test = test['InputUserAnswer']

    mega_p = f"""
      a. We are working with the {db} database, which has the following tables: {tables}

      b. The request to the student is "{question}"

      c. The model answer is {key} (note that we may have multiple correct answers, the model answer is just an example)

      d. The student answer was {test}

      e. Previously, students have submitted these answers and got back these responses and grades: \\n{submissions}

      Please provide a grade and feedback for the student.

      """ + """Only provide a RFC8259 compliant JSON response following this format without deviation. {"grade": "the grade out of 10 you give", "feedback": "the feedback you give"}"""
    return mega_p


## Sample Records from Original DF

In [ ]:
# Randomly select 300 rows from the dataframe
data1 = data.sample(n=300, random_state=1234).reset_index()
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema
0,2028,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 4: Aggregations,"301,300",1,"5,119,728",Count the number of tracks in the database,Long Answer Question,SELECT COUNT(*)\nFROM track;,1.0,cr3431,NaN,SELECT COUNT(*) AS cnt\nFROM track,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
1,909,"Databases for Business Analytics, Spring'23","251,685",Assignment 2B: Selection Queries,"295,300",1,"5,119,647",Show all the albums.,Long Answer Question,select * from album;,1.0,jk7371,NaN,SELECT * \nFROM album,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
2,3308,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 5: Advanced Queries,"301,301",1,"5,119,787","As a first step, limit your analysis only to f...",Long Answer Question,DROP TABLE IF EXISTS eligible_names;\n\nCREATE...,1.0,kgb8897,NaN,DROP TABLE IF EXISTS eligible_names; \nCREATE ...,True,imdb,2.0,NaN,"[{'actors': ['id', 'first_name', 'last_name', ..."
3,292,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 2C: Filtering Queries,"301,294",1,"5,119,680","Find all the tracks that include the word ""Lov...",Long Answer Question,show databases;\nUSE music;\nSHOW Tables;\nSEL...,1.0,soj206,NaN,SELECT *\nFROM track\nWHERE track_name LIKE '%...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na..."
4,1954,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"SELECT CustomerID, Count(OrderID) As NumOrders...",8.0,jrj9504,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,The feedback from the TA was SELECT C.Customer...,"([{'Categories': ['CategoryID', 'CategoryName'..."


In [ ]:
# Generate few-shot mega prompts with submission size = all for GPT4-32k model
lst = []
for i,r in data1.iterrows():
  lst.append(generate_fewshot_prompt_for_row(r))

data1['4_few_mp'] = lst
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,Score,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema,4_few_mp
0,2028,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 4: Aggregations,"301,300",1,"5,119,728",Count the number of tracks in the database,Long Answer Question,SELECT COUNT(*)\nFROM track;,1.0,cr3431,NaN,SELECT COUNT(*) AS cnt\nFROM track,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...
1,909,"Databases for Business Analytics, Spring'23","251,685",Assignment 2B: Selection Queries,"295,300",1,"5,119,647",Show all the albums.,Long Answer Question,select * from album;,1.0,jk7371,NaN,SELECT * \nFROM album,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...
2,3308,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 5: Advanced Queries,"301,301",1,"5,119,787","As a first step, limit your analysis only to f...",Long Answer Question,DROP TABLE IF EXISTS eligible_names;\n\nCREATE...,1.0,kgb8897,NaN,DROP TABLE IF EXISTS eligible_names; \nCREATE ...,True,imdb,2.0,NaN,"[{'actors': ['id', 'first_name', 'last_name', ...",\n a. We are working with the imdb databa...
3,292,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 2C: Filtering Queries,"301,294",1,"5,119,680","Find all the tracks that include the word ""Lov...",Long Answer Question,show databases;\nUSE music;\nSHOW Tables;\nSEL...,1.0,soj206,NaN,SELECT *\nFROM track\nWHERE track_name LIKE '%...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...
4,1954,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"SELECT CustomerID, Count(OrderID) As NumOrders...",8.0,jrj9504,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,The feedback from the TA was SELECT C.Customer...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...


In [ ]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00


In [ ]:
import tiktoken

# counting the number of tokens of mega-prompts
def num_tokens_from_string(string: str, encoding_name="cl100k_base") -> int:
  encoding = tiktoken.get_encoding(encoding_name)
  num_tokens = len(encoding.encode(string))
  return num_tokens

In [ ]:
# Counting the number of tokens for each mega prompt
data1['num'] = data1['4_few_mp'].apply(num_tokens_from_string)
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema,4_few_mp,num
0,2028,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 4: Aggregations,"301,300",1,"5,119,728",Count the number of tracks in the database,Long Answer Question,SELECT COUNT(*)\nFROM track;,...,cr3431,NaN,SELECT COUNT(*) AS cnt\nFROM track,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2899
1,909,"Databases for Business Analytics, Spring'23","251,685",Assignment 2B: Selection Queries,"295,300",1,"5,119,647",Show all the albums.,Long Answer Question,select * from album;,...,jk7371,NaN,SELECT * \nFROM album,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,3348
2,3308,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 5: Advanced Queries,"301,301",1,"5,119,787","As a first step, limit your analysis only to f...",Long Answer Question,DROP TABLE IF EXISTS eligible_names;\n\nCREATE...,...,kgb8897,NaN,DROP TABLE IF EXISTS eligible_names; \nCREATE ...,True,imdb,2.0,NaN,"[{'actors': ['id', 'first_name', 'last_name', ...",\n a. We are working with the imdb databa...,8234
3,292,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 2C: Filtering Queries,"301,294",1,"5,119,680","Find all the tracks that include the word ""Lov...",Long Answer Question,show databases;\nUSE music;\nSHOW Tables;\nSEL...,...,soj206,NaN,SELECT *\nFROM track\nWHERE track_name LIKE '%...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,4640
4,1954,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"SELECT CustomerID, Count(OrderID) As NumOrders...",...,jrj9504,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,The feedback from the TA was SELECT C.Customer...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,6467


In [ ]:
print(data1['num'].max())

17132


In [ ]:
from tqdm import tqdm
import json

In [ ]:
# a function that takes the current row and maximum number of tokens we need
# need improvement since we select training rows randomly
def max_submission_size(t, max_size):
  idf = data[data["QuestionId"] == t['QuestionId']]
  for i in range(1,len(idf)-1):
    mp = generate_fewshot_prompt_for_row(t,i)
    if num_tokens_from_string(mp, "cl100k_base") >= max_size:
      return i-1
  return None

In [ ]:
# 8196 tokens limit for gpt4, select a maximum submission size for each row
lst = []
for i, r in tqdm(data1.iterrows(), total=len(data1)):
  s_size = max_submission_size(r, 6500)
  lst.append(generate_fewshot_prompt_for_row(r,s_size))

data1['4_few_mp'] = lst
data1.head()

100%|██████████| 300/300 [03:03<00:00,  1.63it/s]


,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,NetId,Comment left on User Response,AnswerKey,isQuery,Database,full_score,feedback,schema,4_few_mp,num
0,2028,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 4: Aggregations,"301,300",1,"5,119,728",Count the number of tracks in the database,Long Answer Question,SELECT COUNT(*)\nFROM track;,...,cr3431,NaN,SELECT COUNT(*) AS cnt\nFROM track,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2899
1,909,"Databases for Business Analytics, Spring'23","251,685",Assignment 2B: Selection Queries,"295,300",1,"5,119,647",Show all the albums.,Long Answer Question,select * from album;,...,jk7371,NaN,SELECT * \nFROM album,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,3348
2,3308,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 5: Advanced Queries,"301,301",1,"5,119,787","As a first step, limit your analysis only to f...",Long Answer Question,DROP TABLE IF EXISTS eligible_names;\n\nCREATE...,...,kgb8897,NaN,DROP TABLE IF EXISTS eligible_names; \nCREATE ...,True,imdb,2.0,NaN,"[{'actors': ['id', 'first_name', 'last_name', ...",\n a. We are working with the imdb databa...,6552
3,292,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 2C: Filtering Queries,"301,294",1,"5,119,680","Find all the tracks that include the word ""Lov...",Long Answer Question,show databases;\nUSE music;\nSHOW Tables;\nSEL...,...,soj206,NaN,SELECT *\nFROM track\nWHERE track_name LIKE '%...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,4640
4,1954,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"SELECT CustomerID, Count(OrderID) As NumOrders...",...,jrj9504,"SELECT C.CustomerID, COUNT(O.OrderID) AS order...","SELECT C.CustomerID, COUNT(O.OrderID) AS order...",True,"['northwind', 'flights']",10.0,The feedback from the TA was SELECT C.Customer...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,6467


In [ ]:
data1['num'] = data1['4_few_mp'].apply(num_tokens_from_string)
data1.head()
print(data1['num'].max()) # print the maximum number of tokens among all mega prompts, which is < 8196

7118


In [ ]:
# dealing with rate limit of GPT4
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def GPT4_generation_with_backoff(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        n=1,
        stream=False,
        temperature=0.0,
        max_tokens=600,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
import time
# further control rate limit for GPT-4
rate_limit_per_minute = 40000
delay = 60.0 / rate_limit_per_minute

def delayed_completion(p):
  time.sleep(delay)
  return GPT4_generation_with_backoff(p)

In [ ]:
tqdm.pandas()
# generate GPT4 answer for each mega prompt as a new column of JSON objects
data1['4_answer'] = data1['4_few_mp'].progress_apply(delayed_completion)

100%|██████████| 300/300 [33:25<00:00,  6.69s/it]


In [ ]:
# parse grade and feedback from json object
def extract_grade(js):
  try:
    js = json.loads(js)
    js = js[0]
    return js["grade"]
  except json.JSONDecodeError:
    return ''

def extract_feedback(js):
  try:
    js = json.loads(js)
    js = js[0]
    return js["feedback"]
  except json.JSONDecodeError:
    return ''

In [ ]:
def is_valid_json(json_string):
    try:
        json.loads(json_string)
        return True
    except json.JSONDecodeError:
        return False

for i,r in data1.iterrows():
  if is_valid_json(r['4_answer']) == False:
    print(i)

120


In [ ]:
data1.loc[120,'4_answer']

"Grade: 9/10\n\nFeedback: Your answer is almost perfect. You correctly grouped by the origin and calculated the average fare and average distance. However, for the average fare per mile, you should use the 'fare_per_mile' column directly instead of calculating it as 'fare / distance'. Your query should look like this:\n\nSELECT origin,\n  AVG(fare) AS avg_fare,\n  AVG(fare_per_mile) AS avg_fare_per_mile,\n  AVG(distance) AS avg_distance\nFROM m_ticket_prices\nGROUP BY origin\nORDER BY origin;\n\nKeep up the good work!"

In [ ]:
# Extract grade and feedback as new columns
data1['4_grade'] = data1['4_answer'].apply(extract_grade)
data1['4_feedback'] = data1['4_answer'].apply(extract_feedback)
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,full_score,feedback,schema,4_few_mp,num,4_answer,3_few_mp,4_grade,3_answer,4_feedback
0,2028,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 4: Aggregations,"301,300",1,"5,119,728",Count the number of tracks in the database,Long Answer Question,SELECT COUNT(*)\nFROM track;,...,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2473,"[{""grade"": ""10"", ""feedback"": ""Your answer is c...",\n a. We are working with the music datab...,10,"[{""grade"": ""10"", ""feedback"": ""Great job! Your ...",Your answer is correct. Good job!
1,909,"Databases for Business Analytics, Spring'23","251,685",Assignment 2B: Selection Queries,"295,300",1,"5,119,647",Show all the albums.,Long Answer Question,select * from album;,...,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2498,"[{""grade"": ""10"", ""feedback"": ""Your answer is c...",\n a. We are working with the music datab...,10,"[{""grade"": ""10"", ""feedback"": ""The student's an...",Your answer is correct. Good job!
2,3308,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 5: Advanced Queries,"301,301",1,"5,119,787","As a first step, limit your analysis only to f...",Long Answer Question,DROP TABLE IF EXISTS eligible_names;\n\nCREATE...,...,2.0,NaN,"[{'actors': ['id', 'first_name', 'last_name', ...",\n a. We are working with the imdb databa...,2373,"[{""grade"": ""9.5"", ""feedback"": ""Your answer is ...",\n a. We are working with the imdb databa...,9.5,"[{""grade"": ""1.0"", ""feedback"": ""The student's a...","Your answer is almost perfect. However, the qu..."
3,292,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 2C: Filtering Queries,"301,294",1,"5,119,680","Find all the tracks that include the word ""Lov...",Long Answer Question,show databases;\nUSE music;\nSHOW Tables;\nSEL...,...,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2005,"[{""grade"": ""10"", ""feedback"": ""Your answer is c...",\n a. We are working with the music datab...,10,"[{""grade"": ""1.0"", ""feedback"": ""The query is co...",Your answer is correct. You have successfully ...
4,1954,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"SELECT CustomerID, Count(OrderID) As NumOrders...",...,10.0,The feedback from the TA was SELECT C.Customer...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,2395,"[{""grade"": ""8/10"", ""feedback"": ""Your query is ...",\n a. We are working with the ['northwind...,8/10,"[{""grade"": ""8.0"", ""feedback"": ""The query provi...",Your query is correct and it returns the expec...


In [ ]:
data1['4_grade'].value_counts()

10          211
10/10        38
7.5/10        8
9.5           7
8/10          4
8.0           4
8.5/10        4
5/10          3
0.7           2
9.0           2
1.0           2
7/10          1
6.5/10        1
4.0/10.0      1
0.8           1
7.5           1
9.5/10        1
9.0/10        1
9/10          1
              1
5.0/10        1
7.0/10        1
1.8/2.0       1
5.5/10        1
6.0/10.0      1
3.5/10        1
Name: 4_grade, dtype: int64

In [ ]:
# Repeat the same process for GPT3.5-turbo mega prompt and responses
lst = []
for i, r in tqdm(data1.iterrows(), total=len(data1)):
  s_size = max_submission_size(r, 2500) # max # of tokens = 4096
  lst.append(generate_fewshot_prompt_for_row(r,s_size))

data1['3_few_mp'] = lst

data1['num'] = data1['3_few_mp'].apply(num_tokens_from_string)
print(data1['num'].max())
data1.head()

100%|██████████| 300/300 [01:19<00:00,  3.79it/s]


3064


,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,isQuery,Database,full_score,feedback,schema,4_few_mp,num,4_answer,3_few_mp,4_grade
0,2028,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 4: Aggregations,"301,300",1,"5,119,728",Count the number of tracks in the database,Long Answer Question,SELECT COUNT(*)\nFROM track;,...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2473,"[{""grade"": ""10"", ""feedback"": ""Your answer is c...",\n a. We are working with the music datab...,10
1,909,"Databases for Business Analytics, Spring'23","251,685",Assignment 2B: Selection Queries,"295,300",1,"5,119,647",Show all the albums.,Long Answer Question,select * from album;,...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2498,"[{""grade"": ""10"", ""feedback"": ""Your answer is c...",\n a. We are working with the music datab...,10
2,3308,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 5: Advanced Queries,"301,301",1,"5,119,787","As a first step, limit your analysis only to f...",Long Answer Question,DROP TABLE IF EXISTS eligible_names;\n\nCREATE...,...,True,imdb,2.0,NaN,"[{'actors': ['id', 'first_name', 'last_name', ...",\n a. We are working with the imdb databa...,2373,"[{""grade"": ""9.5"", ""feedback"": ""Your answer is ...",\n a. We are working with the imdb databa...,9.5
3,292,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 2C: Filtering Queries,"301,294",1,"5,119,680","Find all the tracks that include the word ""Lov...",Long Answer Question,show databases;\nUSE music;\nSHOW Tables;\nSEL...,...,True,music,1.0,NaN,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2005,"[{""grade"": ""10"", ""feedback"": ""Your answer is c...",\n a. We are working with the music datab...,10
4,1954,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"SELECT CustomerID, Count(OrderID) As NumOrders...",...,True,"['northwind', 'flights']",10.0,The feedback from the TA was SELECT C.Customer...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,2395,"[{""grade"": ""8/10"", ""feedback"": ""Your query is ...",\n a. We are working with the ['northwind...,8/10


In [ ]:
tqdm.pandas()
data1['3_answer'] = data1['3_few_mp'].progress_apply(GPT3_generation_with_backoff)

100%|██████████| 300/300 [13:06<00:00,  2.62s/it]


In [ ]:
data1['3_grade'] = data1['3_answer'].apply(extract_grade)
data1['3_feedback'] = data1['3_answer'].apply(extract_feedback)
data1.head()

,index,BSpace Course Name,BSpace Course Id,QuizName,﻿QuizId,QuizAttemptNumber,QuestionId,Question,QuestionType,InputUserAnswer,...,schema,4_few_mp,num,4_answer,3_few_mp,4_grade,3_answer,4_feedback,3_grade,3_feedback
0,2028,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 4: Aggregations,"301,300",1,"5,119,728",Count the number of tracks in the database,Long Answer Question,SELECT COUNT(*)\nFROM track;,...,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2473,"[{""grade"": ""10"", ""feedback"": ""Your answer is c...",\n a. We are working with the music datab...,10,"[{""grade"": ""10"", ""feedback"": ""Great job! Your ...",Your answer is correct. Good job!,10,Great job! Your answer is correct and concise.
1,909,"Databases for Business Analytics, Spring'23","251,685",Assignment 2B: Selection Queries,"295,300",1,"5,119,647",Show all the albums.,Long Answer Question,select * from album;,...,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2498,"[{""grade"": ""10"", ""feedback"": ""Your answer is c...",\n a. We are working with the music datab...,10,"[{""grade"": ""10"", ""feedback"": ""The student's an...",Your answer is correct. Good job!,10,The student's answer is correct. They selected...
2,3308,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 5: Advanced Queries,"301,301",1,"5,119,787","As a first step, limit your analysis only to f...",Long Answer Question,DROP TABLE IF EXISTS eligible_names;\n\nCREATE...,...,"[{'actors': ['id', 'first_name', 'last_name', ...",\n a. We are working with the imdb databa...,2373,"[{""grade"": ""9.5"", ""feedback"": ""Your answer is ...",\n a. We are working with the imdb databa...,9.5,"[{""grade"": ""1.0"", ""feedback"": ""The student's a...","Your answer is almost perfect. However, the qu...",1.0,The student's answer is correct and follows th...
3,292,"Databases for Business Analytics, MSQM SU23","277,478",Assignment 2C: Filtering Queries,"301,294",1,"5,119,680","Find all the tracks that include the word ""Lov...",Long Answer Question,show databases;\nUSE music;\nSHOW Tables;\nSEL...,...,"[{'album': ['artist_id', 'album_id', 'album_na...",\n a. We are working with the music datab...,2005,"[{""grade"": ""10"", ""feedback"": ""Your answer is c...",\n a. We are working with the music datab...,10,"[{""grade"": ""1.0"", ""feedback"": ""The query is co...",Your answer is correct. You have successfully ...,1.0,The query is correct and returns the desired r...
4,1954,"Databases for Business Analytics, Winter 2023","251,683",Final Exam,"283,453",1,"4,969,710",For every customer in the database list the nu...,Long Answer Question,"SELECT CustomerID, Count(OrderID) As NumOrders...",...,"([{'Categories': ['CategoryID', 'CategoryName'...",\n a. We are working with the ['northwind...,2395,"[{""grade"": ""8/10"", ""feedback"": ""Your query is ...",\n a. We are working with the ['northwind...,8/10,"[{""grade"": ""8.0"", ""feedback"": ""The query provi...",Your query is correct and it returns the expec...,8.0,The query provided by the student is correct a...


In [ ]:
data1['3_grade'].value_counts()

10          123
1.0          43
7.5          27
9.5          22
0.8          22
8.5          17
0.9           9
10/10         8
0.7           7
9.0           3
8.0           3
5.0           2
1.6           2
2.0           2
8.0/10.0      2
9             2
1.8           2
              1
7.5/10.0      1
10.0          1
0.5           1
Name: 3_grade, dtype: int64

In [ ]:
data1.to_csv('few_shots_completion.csv')